# 利得構造が変動する場合の分析

In [1]:
using Plots: plot, plot!, twinx
using DataFrames: DataFrame
using StatsBase: mean, std
using Random: MersenneTwister

include("../src/Simulation.jl")
using .Simulation: Param, Model, run, POPULATION, PAYOFF, MUTATION
include("../src/Network.jl")
using .Network: nv, degree, neighbors, rem_edge!, normalize_degree!, normalize_weight!

In [2]:
function plot_line(df::DataFrame, skip::Int = 10)::Plots.Plot
    p = plot(df.generation, df.cooperation_rate, label = false, lc = :blue, ylim = (0, 1), yl = "Frequency of Cooperator")
    plot!(twinx(), df.generation, df.N, label = false, lc = :red, ylim = (0, 1000), yl = "N")
    return p
end;

In [3]:
# function debug_log(model::Model)::Nothing
#     cooperation_rate = round(mean(model.strategy_vec .== C), digits = 2)
#     N = size(model.weights, 1)
#     T = round(model.payoff_table_vec[model.generation][(D, C)][1], digits = 2)
#     μ_s_vec = round(model.μ_s_vec[model.generation], digits = 2)
#     μ_c_vec = round(model.μ_c_vec[model.generation], digits = 2)
#     println("generation = $(model.generation), cooperation_rate = $(cooperation_rate), Var = ($(N), $(T), $(μ_s_vec), $(μ_c_vec))")
# end

# function run(param::Param; log_level::Int = 0, log_rate::Float64 = 0.5, log_skip::Int = 10)::DataFrame
#     model = Model(param)

#     start_gen = floor(Int, param.generations * (1 - log_rate)) + 1
#     log_generations = filter(x -> x % log_skip == 0, start_gen:(param.generations))
#     log_row_n = 1

#     output_df = make_output_df(param, log_level, length(log_generations))

#     for generation = 1:(param.generations)
#         model.generation = generation
#         model.payoff_vec .= 0.0

#         interaction!(model)
#         death!(model, param.rng)
#         birth!(model, param.rng)
        
#         normalize_degree!(model.weights, param.initial_k)
#         normalize_weight!(model.weights, param.initial_w * param.initial_k * param.initial_N)

#         if generation ∈ log_generations
#             log!(output_df, model, log_level, log_row_n)
#             log_row_n += 1

#             debug_log(model)
#         end
#     end

#     return output_df
# end;

- なぜ次数が増えるのか？
- いかにして重みを正規化すべきか？(隣人との重みの平均が常にinitial_wになるようにする)
- いかにして次数を正規化すべきか？(initial_k以上になった場合、関係値の低いノードを切断)

In [11]:
param = Param(
    initial_N = 1000,
    initial_k = 100,
    initial_w = 0.5,
    initial_T = 1.7,
    S = -0.7,
    δ = 1.0,
    β = 0.3,
    sigma = 0.01,
    initial_μ_s = 0.01,
    initial_μ_c = 0.01,
    generations = 1000,
    variability_mode = MUTATION,
    rng = MersenneTwister(1),
)
@time output_df1 = run(param, log_rate = 1.0, log_skip = 10)

 11.642652 seconds (34.41 M allocations: 32.427 GiB, 8.57% gc time)


Row,initial_N,initial_k,initial_T,S,initial_w,Δw,interaction_freqency,reproduction_rate,δ,initial_μ_s,initial_μ_c,β,sigma,generations,variability_mode,generation,N,T,cooperation_rate,payoff_μ
,Int64,Int64,Float64,Float64,Float16,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,String,Int16,Int16,Float16,Float16,Float16
1,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,10,1000,1.7,0.006,0.0111
2,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,20,1000,1.7,0.014,0.0267
3,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,30,1000,1.7,0.009,0.019
4,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,40,1000,1.7,0.013,0.0237
5,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,50,1000,1.7,0.009,0.0239
6,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,60,1000,1.7,0.009,0.0187
7,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,70,1000,1.7,0.011,0.0274
8,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,80,1000,1.7,0.01,0.0246
9,1000,100,1.7,-0.7,0.5,0.1,1.0,0.1,1.0,0.01,0.01,0.3,0.01,1000,MUTATION,90,1000,1.7,0.011,0.0284


In [5]:
# param = Param(
#     initial_N = 500,
#     initial_k = 100,
#     initial_w = 0.5,
#     initial_T = 1.1,
#     S = -0.1,
#     δ = 1.0,
#     β = 0.5,
#     sigma = 300,
#     initial_μ_s = 0.01,
#     initial_μ_c = 0.01,
#     generations = 1000,
#     variability_mode = POPULATION,
# )
# @time output_df2 = run(param)

In [6]:
# param = Param(
#     initial_N = 500,
#     initial_k = 498,
#     initial_w = 0.5,
#     initial_T = 1.1,
#     S = -0.1,
#     δ = 1.0,
#     β = 0.5,
#     sigma = 300,
#     initial_μ_s = 0.01,
#     initial_μ_c = 0.01,
#     generations = 1000,
#     variability_mode = POPULATION,
# )
# @time output_df3 = run(param)

In [7]:
# plot(plot_line(output_df1), plot_line(output_df2), plot_line(output_df3), layout = (1, 3), size = (1200, 300))

In [8]:
# mean(output_df1.cooperation_rate),
# mean(output_df2.cooperation_rate),
# mean(output_df3.cooperation_rate)